<a href="https://colab.research.google.com/github/arthurwelle/Cecon/blob/master/Copy_of_Aula_Expositiva.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# POF: conceitos utilizados, estrutura, limites e potencialidades

**24/04/2019**


## Introdução à POF

<u>Referência:</u> Para entender a POF (Gaiger)

Nota-se uma convergência ao estudo de tributação utilizando a POF. Informações de consumo e padrões de consumo são bastante ricas. Isso pode ser visto que 12/16 arquivos da POF relacianadas ao consumo

Gaiger pontua a necessidade de unir serviços adminitrativos com a POF. Deficiências da POF:

- Ausência de orçamentos financeiros das famílias
  - Famílias reportam déficit orçamentário
  - Destaque para mudança no acesso ao crédito
- Desemprego e atividade secundária (ver PNAD) nem horas de trabalho
- Crítica de inputação dos dados e quantidade
- Descontinuidades
- **Principal objetivo:** Índice de preços
- Sem rendimentos de aposentadorias discriminados
- Pouco estudos sobre fontes de rendimentos (não-trabalho e não-aposentadoria)

## Características

Pesquisa de duração de um ano que são realizadas em uma semana em determinada família para coletar informações relacionadas a consumo. Distinção entre famílias e unidade de consumo que, em termos práticos, não é bastante significativa uma vez que são raros os casos de famílias com duas unidades de consumo. Além disso, são selecionados quem responderá o questionário sobre gastos individuais (maiores de 10 anos) e do domicílio. A unidade de entrada é a aquisição não necessariamente consumidos (distinção entre consumo e gasto). Vale destacar que o cálculo de linha de pobreza estão relacionadas com o tipo de consumo mas existem estimativas multidimensionais.

## Baixando dados

In [0]:
!rm *.txt # Apaga arquvios antigos
!rm -rf sample_data # Apaga pasta 
!wget ftp://ftp.ibge.gov.br/Orcamentos_Familiares/Pesquisa_de_Orcamentos_Familiares_2008_2009/Microdados/Dados.zip #Baixa
!unzip -u Dados.zip # Deszipa
!rm Dados.zip # Deleta arquivo .zip
!apt-get install p7zip # Instala programa para descompactar .7z
!p7zip -d  Dados/*.7z # Descompacta .7z
!rm -rf Dados

--2019-04-24 14:45:18--  ftp://ftp.ibge.gov.br/Orcamentos_Familiares/Pesquisa_de_Orcamentos_Familiares_2008_2009/Microdados/Dados.zip
           => ‘Dados.zip’
Resolving ftp.ibge.gov.br (ftp.ibge.gov.br)... 170.84.40.44
Connecting to ftp.ibge.gov.br (ftp.ibge.gov.br)|170.84.40.44|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /Orcamentos_Familiares/Pesquisa_de_Orcamentos_Familiares_2008_2009/Microdados ... done.
==> SIZE Dados.zip ... 59098084
==> PASV ... done.    ==> RETR Dados.zip ... done.
Length: 59098084 (56M) (unauthoritative)

Dados.zip           100%[===================>]  56.36M   909KB/s    in 48s     

2019-04-24 14:46:09 (1.18 MB/s) - ‘Dados.zip’ saved [59098084]

Archive:  Dados.zip
  inflating: Dados/T_ALUGUEL_ESTIMADO_S.7z  
  inflating: Dados/T_CADERNETA_DESPESA_S.7z  
  inflating: Dados/T_CONDICOES_DE_VIDA_S.7z  
  inflating: Dados/T_CONSUMO_S.7z    
  inflating: Dados/T_DESPESA_12MESE

In [0]:
!wget ftp://ftp.ibge.gov.br/Orcamentos_Familiares/Pesquisa_de_Orcamentos_Familiares_2008_2009/Microdados/tradutores.zip #Baixa
!mkdir tradutores
!unzip -u tradutores.zip  -o tradutores # Deszipa 
!rm tradutores.zip # Deleta arquivo .zip

--2019-04-24 14:46:28--  ftp://ftp.ibge.gov.br/Orcamentos_Familiares/Pesquisa_de_Orcamentos_Familiares_2008_2009/Microdados/tradutores.zip
           => ‘tradutores.zip’
Resolving ftp.ibge.gov.br (ftp.ibge.gov.br)... 170.84.40.44
Connecting to ftp.ibge.gov.br (ftp.ibge.gov.br)|170.84.40.44|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /Orcamentos_Familiares/Pesquisa_de_Orcamentos_Familiares_2008_2009/Microdados ... done.
==> SIZE tradutores.zip ... 83876
==> PASV ... done.    ==> RETR tradutores.zip ... done.
Length: 83876 (82K) (unauthoritative)

tradutores.zip      100%[===================>]  81.91K   132KB/s    in 0.6s    

2019-04-24 14:46:32 (132 KB/s) - ‘tradutores.zip’ saved [83876]

mkdir: cannot create directory ‘tradutores’: File exists
Archive:  tradutores.zip
caution: filename not matched:  -o
caution: filename not matched:  tradutores


# Utilizando pacote R

In [0]:
%load_ext rpy2.ipython
import warnings
warnings.filterwarnings('ignore')

In [0]:
%%R
install.packages( "devtools" , repos = "http://cran.rstudio.com/" )
library(devtools)
install_github( "ajdamico/lodown" , dependencies = TRUE )
install.packages( "convey" , repos = "http://cran.rstudio.com/" )
install.packages( "srvyr" , repos = "http://cran.rstudio.com/" )
library(lodown)
library(convey)
library(srvyr)

In [0]:
%%R
pof_cat <-
    get_catalog( "pof" ,
        output_dir = file.path( path.expand( "~/content" ) , "POF" ) )

# 2008-2009 only
pof_cat <- subset( pof_cat , period == "2008_2009" )
# download the microdata to your local computer
pof_cat <- lodown( "pof" , pof_cat )

building catalog for pof

locally downloading pof

downloading from URL
'ftp://ftp.ibge.gov.br/Orcamentos_Familiares/Pesquisa_de_Orcamentos_Familiares_2008_2009/Microdados/Dados.zip'
to file
'/tmp/Rtmp3p66tn/file14ba1994c02e'

download issue with
'ftp://ftp.ibge.gov.br/Orcamentos_Familiares/Pesquisa_de_Orcamentos_Familiares_2008_2009/Microdados/Dados.zip'

download issue with
'ftp://ftp.ibge.gov.br/Orcamentos_Familiares/Pesquisa_de_Orcamentos_Familiares_2008_2009/Microdados/Dados.zip'

download issue with
'ftp://ftp.ibge.gov.br/Orcamentos_Familiares/Pesquisa_de_Orcamentos_Familiares_2008_2009/Microdados/Dados.zip'

R version 3.5.3 (2019-03-11)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Ubuntu 18.04.2 LTS

Matrix products: default
BLAS: /usr/lib/x86_64-linux-gnu/openblas/libblas.so.3
LAPACK: /usr/lib/x86_64-linux-gnu/openblas/liblapack.so.3

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MO

In [0]:
%R -o pof_cat

In [0]:
pof_cat['full_urls']

0    ftp://ftp.ibge.gov.br/Orcamentos_Familiares/Pe...
Name: full_urls, dtype: object